### **5. DR-learner**

DR-learner is a two-stage doubly robust estimator for HTE estimation. Before Kennedy et al. 2020 [4], there are several related approaches trying to extend the doubly robust procedure to HTE estimation, such as [5, 6, 7]. Compared with the above three estimators, DR-learner is proved to be oracle efficient under some mild assumptions detailed in Theorem 2 of [4].

The basic steps of DR-learner is given below:

**Step 1**: Nuisance training: \\
(a)  Using $I_{1}^n$ to construct estimates $\hat{\pi}$ for the propensity scores $\pi$; \\
(b)  Using $I_{1}^n$ to construct estimates $\hat\mu_a(s)$ for $\mu_a(s):=\mathbb{E}[R|S=s,A=a]$;

**Step 2**: Pseudo-outcome regression: \\
Define $\widehat{\phi}(Z)$ as the pseudo-outcome where 
\begin{equation}
\widehat{\phi}(Z)=\frac{A-\hat{\pi}(S)}{\hat{\pi}(S)\{1-\hat{\pi}(S)\}}\Big\{R-\hat{\mu}_A(S)\Big\}+\hat{\mu}_1(S)-\hat{\mu}_0(S),
\end{equation}
and regress it on covariates $S$ in the test sample $I_2^n$, yielding 
\begin{equation}
\widehat{\tau}_{\text{DR-learner}}(s)=\widehat{\mathbb{E}}_n[\widehat{\phi}(Z)|S=s].
\end{equation}


In [1]:
import sys
!{sys.executable} -m pip install scikit-uplift

In [2]:
# import related packages
from matplotlib import pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression 
from causaldm._util_causaldm import *
from causaldm.learners.Causal_Effect_Learning.Single_Stage.DRlearner import DRlearner

OSError: dlopen(/Users/alinaxu/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so, 0x0006): Library not loaded: '/usr/local/opt/libomp/lib/libomp.dylib'
  Referenced from: '/Users/alinaxu/opt/anaconda3/lib/python3.9/site-packages/lightgbm/lib_lightgbm.so'
  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)

In [ ]:
n = 10**3  # sample size in observed data
n0 = 10**5 # the number of samples used to estimate the true reward distribution by MC
seed=223

In [ ]:
# Get data
data_behavior = get_data_simulation(n, seed, policy="behavior")
#data_target = get_data_simulation(n0, seed, policy="target")

# The true expected heterogeneous treatment effect
HTE_true = get_data_simulation(n, seed, policy="1")['R']-get_data_simulation(n, seed, policy="0")['R']



In [ ]:
# DR-learner for HTE estimation
outcome = 'R'
treatment = 'A'
controls = ['S1','S2']
n_folds = 5
y_model = LGBMRegressor(max_depth=2)
ps_model = LogisticRegression()
Rlearner_model = LGBMRegressor(max_depth=2)

HTE_DR_learner = DRlearner(data_behavior, outcome, treatment, controls, y_model, ps_model)
HTE_DR_learner = HTE_DR_learner.to_numpy()

estimate with DR-learner
fold 1, testing r2 baselearner: 0.980, pslearner: 0.943
fold 2, testing r2 baselearner: 0.978, pslearner: 0.947
fold 3, testing r2 baselearner: 0.975, pslearner: 0.942
fold 4, testing r2 baselearner: 0.978, pslearner: 0.946
fold 5, testing r2 baselearner: 0.978, pslearner: 0.940


In [ ]:
print("DR-learner:  ",HTE_DR_learner[0:8])
print("true value: ",HTE_true[0:8].to_numpy())

DR-learner:   [-1.2566  0.0408 -0.8131 -0.0906 -0.5665 -0.7341 -0.6459 -1.272 ]
true value:  [ 1.2961 -0.4475  0.731   0.2863  0.4471 -0.1839 -3.3869 -1.238 ]


In [ ]:
Bias_DR_learner = np.sum(HTE_DR_learner-HTE_true)/n
Variance_DR_learner = np.sum((HTE_DR_learner-HTE_true)**2)/n
print("The overall estimation bias of DR-learner is :     ", Bias_DR_learner, ", \n", "The overall estimation variance of DR-learner is :",Variance_DR_learner,". \n")

The overall estimation bias of DR-learner is :      0.29436318987432813 , 
 The overall estimation variance of DR-learner is : 4.011818461500106 . 



## References

2. Xinkun Nie and Stefan Wager. Quasi-oracle estimation of heterogeneous treatment effects. Biometrika, 108(2):299–319, 2021.

3. Peter M Robinson. Root-n-consistent semiparametric regression. Econometrica: Journal of the Econometric Society, pages 931–954, 1988.

4. Edward H Kennedy. Optimal doubly robust estimation of heterogeneous causal effects. arXiv preprint arXiv:2004.14497, 2020

5. M. J. van der Laan. Statistical inference for variable importance. The International Journal of Biostatistics, 2(1), 2006.

6. S. Lee, R. Okui, and Y.-J. Whang. Doubly robust uniform confidence band for the conditional average treatment effect function. Journal of Applied Econometrics, 32(7):1207–1225, 2017.

7. D. J. Foster and V. Syrgkanis. Orthogonal statistical learning. arXiv preprint arXiv:1901.09036, 2019.